# Lab 1: MNIST

In [14]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms # MNIST, CIFAR10/100, ImageNet12, COCO, LSUN???
from torch.autograd import Variable

# Training settings
no_cuda = False
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 100

use_cuda = not no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# Add data augmentation into train_loader and test_loader. Composes several transforms together.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)) # mean, std
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

Processing...
Done!


In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320) # unlike reshape, view share underlying data with previous variable
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x#F.log_softmax(x)

In [18]:
model = Net()
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [15]:
# use log_softmax & negative log likelihood

def train(epoch):
    model.train() # set the network as train mode and this only affect things like batch-normalization or dropout
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data) # forward function???
        loss = F.nll_loss(output, target) # negative log likelihood loss
        loss.backward() # update params in this step or next step???
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.154914
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.243720
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.027876
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.082032
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.246461
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.162370
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.054578
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.178934
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.076694
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.071996

Test set: Average loss: 0.0408, Accuracy: 9879/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.297944
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.098082
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.101063
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.149843
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.215960
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.092185
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.213296
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.295363
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.293098
T

In [19]:
# use cross-entropy

def train(epoch):
    model.train() # set the network as train mode and this only affect things like batch-normalization or dropout
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data) # forward function???
        #loss = F.nll_loss(output, target) # negative log likelihood loss
        loss = F.cross_entropy(output, target)
        loss.backward() # update params in this step or next step???
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        #test_loss += F.nll_loss(output, target).data[0]
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.289578
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.648514
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.086706
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.898719
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.676648
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.623288
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.593553
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.351258
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.501225
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.382125

Test set: Average loss: 0.1916, Accuracy: 9464/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.370484
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.360386
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.541314
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.243724
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.579458
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.400228
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.364568
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.398022
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.444082
T